In [26]:
# 1. Importation des bibliothèques nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
import os
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

# 2. Chargement des données
train_df = pd.read_csv('/home/charlemagne/workspace/kaggle_challenge_titanic/data/train.csv')
test_df = pd.read_csv('/home/charlemagne/workspace/kaggle_challenge_titanic/data/test.csv')
submission_file_path = '/home/charlemagne/workspace/kaggle_challenge_titanic/submission/submission.csv'

train_df.head(10)

def preprocess(df):
    df = df.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
        
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)                     
    return df
    
preprocessed_train_df = preprocess(train_df)
preprocessed_test_df = preprocess(test_df)
preprocessed_train_df.head(5)

input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")
#input_features.remove("Ticket_number")

print(f"Input features: {input_features}")

def tokenize_names(features, labels=None):
    """Divite the names into tokens. TF-DF can consume text tokens natively."""
    features["Name"] =  tf.strings.split(features["Name"])
    return features, labels

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_train_df,label="Survived").map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_test_df).map(tokenize_names)

model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0, # Very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True, # Only use the features in "features"
    random_seed=1234,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0,
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,
    min_examples=1,
    categorical_algorithm="RANDOM",
    shrinkage=0.05,
    split_axis="SPARSE_OBLIQUE",
    sparse_oblique_normalization="MIN_MAX",
    sparse_oblique_num_projections_exponent=2.0,
    num_trees=2000,
    random_seed=1234,
    
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss:{self_evaluation.loss}")

model.summary()

def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(serving_ds, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": preprocessed_test_df["PassengerId"],
        "Survived": (proba_survive >= threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    kaggle_predictions.to_csv(submission_file_path, index=False)
    print(f"Submission exported to {submission_file_path}")
    
kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)

tuner = tfdf.tuner.RandomSearch(num_trials=1000)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                     ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=0)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss:{tuned_self_evaluation.loss}")

predictions = None
num_predictions = 0

for i in range(100):
    print(f"i:{i}")
    model = tfdf.keras.GradientBoostedTreesModel(
        verbose=0, 
        features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
        exclude_non_specified_features=True, # Only use the features in "features"
        random_seed=i,
        honest=True,
    )
    model.fit(train_ds)
    
    sub_predictions = model.predict(serving_ds, verbose=0)[:,0]
    if predictions is None:
        predictions = sub_predictions
    else:
        predictions += sub_predictions
    num_predictions += 1

predictions/=num_predictions

kaggle_predictions = pd.DataFrame({
        "PassengerId": preprocessed_test_df["PassengerId"],
        "Survived": (predictions >= 0.5).astype(int)
    })
submission = kaggle_predictions.copy()  # Create a copy of kaggle_predictions
submission.to_csv(submission_file_path, index=False)


Input features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


[WARNING 24-01-26 19:52:51.4200 CET gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-26 19:52:51.4200 CET gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-26 19:52:51.4200 CET gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-01-26 19:52:51.6863 CET kernel.cc:1233] Loading model from path /tmp/tmpnqyibbhn/model/ with prefix 6dd9f14d8455443f
[INFO 24-01-26 19:52:51.6877 CET quick_scorer_extended.cc:903] The binary was compiled without AVX2 support, but your CPU supports it. Enable it for faster model inference.
[INFO 24-01-26 19:52:51.6879 CET abstract_model.cc:1344] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 24-01-26 19:52:51.6879 CET kernel.cc:1061] Use fast generic engine
[WARNING 24-01-26 19:52:51.8149 CET gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" no

Accuracy: 0.8260869383811951 Loss:0.8608942627906799
Accuracy: 0.782608687877655 Loss:1.0586705207824707
Model: "gradient_boosted_trees_model_108"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	Age
	Cabin
	Embarked
	Fare
	Name
	Parch
	Pclass
	Sex
	SibSp
	Ticket_item
	Ticket_number

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.           "Sex"  0.585997 ################
    2.           "Age"  0.364636 #######
    3.          "Fare"  0.266191 ###
    4.          "Name"  0.207054 #
    5.        "Pclass"  0.179191 
    6. "Ticket_number"  0.178806 
    7.      "Embarked"  0.177803 
    8.   "Ticket_item"  0.177009 
    9.         "Parch"  0.

[INFO 24-01-26 19:52:52.1737 CET kernel.cc:1233] Loading model from path /tmp/tmpoxz2y2d9/model/ with prefix 32f61bfc377f41b4
[INFO 24-01-26 19:52:52.1762 CET decision_forest.cc:660] Model loaded with 40 root(s), 2106 node(s), and 10 input feature(s).
[INFO 24-01-26 19:52:52.1763 CET kernel.cc:1061] Use fast generic engine
[WARNING 24-01-26 19:52:52.3599 CET gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-26 19:52:52.3599 CET gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-26 19:52:52.3599 CET gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-01-26 19:53:04.2191 CET kernel.cc:1233] Loading model from path /tmp/tmpnh12yh42/model/ with prefix 41137c35a73c4b08
[INFO 24-01-26 19:53:04.2265 CET decision_forest.cc:660] Model loaded with 30 root(s), 2236 node(s), and 12 input feature(s).
[INFO 24-01-26 1

Accuracy: 0.8630136847496033 Loss:0.6749962568283081
i:0


...

[INFO 24-01-26 19:53:49.5414 CET kernel.cc:1233] Loading model from path /tmp/tmpiro40z31/model/ with prefix 52dc0fe8b6b5476d
[INFO 24-01-26 19:53:49.5427 CET kernel.cc:1061] Use fast generic engine
[WARNING 24-01-26 19:53:49.7046 CET gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-26 19:53:49.7047 CET gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-26 19:53:49.7047 CET gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


i:98


[INFO 24-01-26 19:53:49.9651 CET kernel.cc:1233] Loading model from path /tmp/tmphz7nd0s5/model/ with prefix d2e3d78fd5194d32
[INFO 24-01-26 19:53:49.9668 CET kernel.cc:1061] Use fast generic engine
[WARNING 24-01-26 19:53:50.1344 CET gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-26 19:53:50.1344 CET gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-26 19:53:50.1344 CET gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


i:99


[INFO 24-01-26 19:53:50.4814 CET kernel.cc:1233] Loading model from path /tmp/tmp07roq87n/model/ with prefix 4b0d30f765a64fc3
[INFO 24-01-26 19:53:50.4846 CET kernel.cc:1061] Use fast generic engine
